# Validation - BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import multimodalfitting as mf

# %matplotlib notebook
%matplotlib widget

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = Path("..") / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
result_folder = Path("..") / "results" / '211124' 
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
ais_recordings = {}

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

# probe = mf.define_electrode(probe_type=probe_type)
probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
params_release

# Test calcium spike

In [ ]:
best_soma = np.argmin(opt_soma.best_fitness)
best_sample_soma = opt_soma.iloc[best_soma]
best_params_soma_dict = {k: v for k, v in zip(param_names, best_sample_soma.best_params)}
display(best_sample_soma.best_fitness)

opt_all = opt_extra.query("extra_strategy == 'all'")
opt_sections = opt_extra.query("extra_strategy == 'sections'")
opt_single = opt_extra.query("extra_strategy == 'single'")

best_all = np.argmin(opt_all.best_fitness)
best_sections = np.argmin(opt_sections.best_fitness)
best_single = np.argmin(opt_single.best_fitness)

best_sample_all = opt_all.iloc[best_all]
best_params_all_dict = {k: v for k, v in zip(param_names, best_sample_all.best_params)}
display("ALL", best_sample_all.best_fitness)

best_sample_sections = opt_sections.iloc[best_sections]
best_params_sections_dict = {k: v for k, v in zip(param_names, best_sample_sections.best_params)}
display("SECTIONS", best_sample_sections.best_fitness)

best_sample_single = opt_single.iloc[best_single]
best_params_single_dict = {k: v for k, v in zip(param_names, best_sample_single.best_params)}
display("SINGLE", best_sample_single.best_fitness)

In [ ]:
kwargs = {"syn_weight": 2, "pulse_amp": 1}

In [ ]:
print("Computing RELEASE")
response_release = mf.utils.simulate_BAC_responses(cell, params_release, sim, **kwargs)
print("Computing SOMA")
response_soma = mf.utils.simulate_BAC_responses(cell, best_params_soma_dict, sim, **kwargs)
print("Computing ALL")
response_all = mf.utils.simulate_BAC_responses(cell, best_params_all_dict, sim, **kwargs)
print("Computing SECTIONS")
response_sections = mf.utils.simulate_BAC_responses(cell, best_params_sections_dict, sim, **kwargs)
print("Computing SINGLE")
response_single = mf.utils.simulate_BAC_responses(cell, best_params_single_dict, sim, **kwargs)

In [ ]:
ax_dict = {}

loc_names = response_release.keys()

for loc in loc_names:
    fig, ax = plt.subplots()
    ax_dict[loc] = ax
    
    for i, resp_name in enumerate(response_release):
        resp = response_release
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="k", label="GT")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="k")

    for i, resp_name in enumerate(response_soma):
        resp = response_soma
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0", label="soma")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0")

    for i, resp_name in enumerate(response_all):
        resp = response_all
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1", label="all")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1")
    
    for i, resp_name in enumerate(response_sections):
        resp = response_sections
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C2", label="sections")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C2")
    
    for i, resp_name in enumerate(response_single):
        resp = response_single
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C3", label="single")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C3")

    ax.set_title(f"Recording from {loc}")
    ax.legend()